# NVIDIA RAG Blueprint with Pinecone - Quickstart

This notebook demonstrates how to:
1. Set up the document processing pipeline
2. Ingest documents using Nemo Retriever Extract
3. Store vectors in Pinecone
4. Perform RAG queries

## Setup

First, ensure you have the required environment variables:

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify required variables
required_vars = [
    'NVIDIA_API_KEY',
    'PINECONE_API_KEY',  # Only for cloud deployment
    'PINECONE_ENVIRONMENT'  # Only for cloud deployment
]

for var in required_vars:
    if not os.getenv(var):
        print(f'Warning: {var} not set')

## Initialize Document Processor

Set up the document processor using Nemo Retriever Extract:

In [ ]:
from src.document_processor import DocumentProcessor
from src.pinecone_uploader import PineconeUploader

# Initialize processors
doc_processor = DocumentProcessor(
    embedding_model_name="nvidia/llama-3.2-nv-embedqa-1b-v2",
    extract_config={
        "extract_text": True,
        "extract_tables": True,
        "extract_charts": True,
        "extract_images": True,
        "text_depth": "page"
    }
)

pinecone_uploader = PineconeUploader()

## Process Sample Documents

Let's process some sample documents from the NVIDIA Developer Blog:

In [ ]:
# Process documents
data_dir = "../data/sample_docs"
processed_docs = doc_processor.process_documents(data_dir)

print(f"Processed {len(processed_docs)} documents")

# Upload to Pinecone
pinecone_uploader.upload_vectors(processed_docs)
print("Vectors uploaded to Pinecone")

## Perform RAG Queries

Now let's try some queries using our RAG pipeline:

In [ ]:
from src.retriever import NVIDIARetriever

# Initialize retriever
retriever = NVIDIARetriever()

# Example query
query = "What are the key benefits of using NVIDIA's RAG blueprint?"

# Get relevant documents
results = await retriever.retrieve(query, top_k=3)

print("\nRelevant passages:")
for i, result in enumerate(results, 1):
    print(f"\n{i}. Score: {result['score']:.3f}")
    print(f"Text: {result['metadata']['text'][:200]}...")

## Cleanup

If you're using Pinecone Local, remember that data is not persistent and will be cleared when the service restarts.